## THE ADVISORY FILE


## loading the datasets

In [ ]:
import pandas as pd
import requests

try:
    df = pd.read_csv('crop_recommendation.csv')
    print(f"Dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'crop_recommendation.csv' not found. Please check the file path.")

def test_api_connection(api_key):
    test_url = f"https://api.openweathermap.org/data/2.5/weather?q=London&appid={api_key}"
    try:
        response = requests.get(test_url, timeout=5)
        if response.status_code == 200:
            print("API Connection: Success! Key is valid.")
            return True
        elif response.status_code == 401:
            print("API Connection: Failed! Invalid API Key.")
        else:
            print(f"API Connection: Unexpected status {response.status_code}")
        return False
    except requests.exceptions.RequestException as e:
        print(f"Network Error: Could not connect to API. {e}")
        return False

def get_location_name(lat, lon):
    api_key = "d16ca93ca7c89a66afa56bd31a522391"
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key,
        'units': 'metric'
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status() 
        
        data = response.json()
        return data.get('name', 'Unknown Location')

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    
    return 'Unknown Location'

# TEST THE API
MY_API_KEY = "d16ca93ca7c89a66afa56bd31a522391"
if test_api_connection(MY_API_KEY):
    # sample coordinate (Nyeri)
    location = get_location_name(-0.42, 36.95)
    print(f"Tested location retrieval: {location}")

## months that favor the crops

In [ ]:

crop_months = {
   'rice': [3, 4, 5, 10, 11], 'maize': [3, 4, 9, 10], 'chickpea': [1, 2, 11, 12],
    'kidneybeans': [5, 6], 'pigeonpeas': [6, 7], 'mothbeans': [7, 8],
    'mungbean': [8, 9], 'blackgram': [6, 7, 8], 'lentil': [1, 2, 11, 12],
    'pomegranate': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'banana': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'mango': [1, 2, 3], 'grapes': [11, 12, 1, 2], 'watermelon': [1, 2, 3],
    'muskmelon': [1, 2, 3], 'apple': [1, 2, 11, 12], 'orange': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'papaya': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'coconut': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'cotton': [5, 6, 7, 8], 'jute': [3, 4, 5], 'coffee': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
}

print("crop months set!")


### risk assessment layer

In [ ]:
def risk_assessment ( humidity, rainfall, temperature):
    risk_score = 0
    factors = []

    #fungal attack is high when humidity is high
    if humidity > 80 and 20 <= temperature<=30:
        risk_score += 60
        factors.append("Heavy rainfall and warmth: Your crops are at risk of fungal blight.")
    if rainfall>200:
        risk_score += 20
        factors.append("Heavy rainfall: Monitor for water-borne pests.")
    if rainfall <20 and temperature >35:
        risk_score +=70
        factors.append("Extreme heat and low rainfall amounts: Your crops are at high risk of wilting")
    #categorixing the risk
    level = "low"
    if risk_score >70 : level = "high"
    elif risk_score > 40: level = 'medium'


    return {"level": level, "score": risk_score, "warnings": factors}


### monthly data usage

In [ ]:
import requests
import pandas as pd
import datetime

def test_openmeteo_health():
    test_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": 0, "longitude": 0, 
        "start_date": "2023-01-01", "end_date": "2023-01-01", 
        "daily": "temperature_2m_mean"
    }
    try:
        res = requests.get(test_url, params=params, timeout=5)
        if res.status_code == 200:
            print("Open-Meteo API: Online and reachable.")
            return True
        else:
            print(f"Open-Meteo API: Returned status {res.status_code}. Data may be unavailable.")
            return False
    except Exception as e:
        print(f" Open-Meteo API: Connection failed. {e}")
        return False

def openmeteo_monthly_weather(lat, lon, year=None):
    today = datetime.datetime.now()
    if year is None:
        # Using last year since I want a complete annual record
        year = today.year - 1 
    
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation"],
        "timezone": "Africa/Nairobi"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        
        response.raise_for_status() 
        data = response.json()

        # Does the response actually have the expected keys
        if 'hourly' not in data:
            print(f"API Warning: No hourly data found for coordinates ({lat}, {lon}).")
            return 25.0, 70.0, 50.0, "Unknown"

        #DataFrame 
        df_weather = pd.DataFrame({
            'time': pd.to_datetime(data['hourly']['time']),
            'temperature': data['hourly']['temperature_2m'],
            'humidity': data['hourly']['relative_humidity_2m'],
            'rainfall': data['hourly']['precipitation']
        })

        df_weather.set_index('time', inplace=True)

        # Monthly aggregation  >>> 'ME' (Month End)
        monthly_data = df_weather.resample('ME').agg({
            'temperature': 'mean',
            'humidity': 'mean',
            'rainfall': 'sum'
        })

        # Get the specific month matching current month
        target_month_index = today.month - 1
        if target_month_index < len(monthly_data):
            latest = monthly_data.iloc[target_month_index]
            return latest['temperature'], latest['humidity'], latest['rainfall'], "Success"
        else:
            return monthly_data.iloc[-1]['temperature'], monthly_data.iloc[-1]['humidity'], monthly_data.iloc[-1]['rainfall'], "Success"

    except requests.exceptions.RequestException as e:
        print(f" Network Error: {e}")
    except Exception as e:
        print(f"Processing Error: {e}")
    
    # Fallback to defaults to prevent the whole app from crashing
    return 25.0, 70.0, 100.0, "Fallback used"

# --- EXECUTION ---
if test_openmeteo_health():
    temp, hum, rain, status = openmeteo_monthly_weather(-0.42, 36.95)
    print(f"Status: {status} | Temp: {temp:.2f}°C, Hum: {hum:.2f}%, Rain: {rain:.2f}mm")

## training and testing the datasets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

#TRAINING
X= df.drop('label', axis=1) # X = FEATURE (QUESTION / INPUTS)
y=df['label']    #Y = TARGET (ANSWER)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.head)

#training only the training set
model =RandomForestClassifier(n_estimators=100) 
model.fit(X_train, y_train)

#evaluating the test
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('Model training complete!')
print(f"RANDOM FOREST ACCURACY SCORE: {accuracy * 100:.2f}%")


##  The advisory logic

In [ ]:
import joblib
import datetime


CROP_RANGES = df.groupby('label').agg(['min','max']).to_dict()
def check_crop_viability(crop_name, temperature, humidity, ph, rainfall):
    crop_name = crop_name.lower().strip()
    
    if crop_name not in df['label'].unique():
        return False, f"Sorry, '{crop_name}' is not in our recommendation database, We are working on enlarging the crop database, keep in touch as we work on our end. Thank you for your patience"

    issues = []
    
    def is_outside(val, col):
        low = CROP_RANGES[(col, 'min')][crop_name]
        high = CROP_RANGES[(col, 'max')][crop_name]
        if val < low: return f"too low (ideal: {low:.1f}-{high:.1f})"
        if val > high: return f"too high (ideal: {low:.1f}-{high:.1f})"
        return None

    checks = {
        'temperature': temperature,
        'humidity': humidity,
        'ph': ph,
        'rainfall': rainfall
    }

    for label, value in checks.items():
        err = is_outside(value, label)
        if err:
            issues.append(f"{label.capitalize()} is {err}")

    if not issues:
        return True, f"Conditions are perfect! {crop_name.capitalize()} thrives in these settings."
    else:
        return False, f"Warning for {crop_name}: " + " | ".join(issues)


def farmer_advice(lat, lon, soil_type, n, p, k, ph, user_crop=None):
    location_name = get_location_name(lat, lon)
    #current_month = datetime.datetime.now().month

    temperature, humidity, rainfall, _ = openmeteo_monthly_weather(lat, lon)

    input_df = pd.DataFrame([{
        'N': n,
        'P': p,
        'K': k,
        'temperature': temperature,
        'humidity': humidity,
        'ph': ph,
        'rainfall': rainfall
    }])
    #prediction
    prediction = model.predict(input_df)
    risk_info = risk_assessment(temperature, humidity, rainfall)

    month_name= datetime.datetime.now().strftime("%B")

    #HANDLE USER CROP CHOICE
    viability_msg = ""
    if user_crop:
        is_viable, viability_msg = check_crop_viability(user_crop, temperature, humidity, ph, rainfall)

    final_recommendation = user_crop if user_crop else prediction
    advice = {
        "location": f"{location_name} ({lat}, {lon})",
        "recommended_crop": prediction[0],
        "risk_level": risk_info["level"],
        "soil_analysis": f"Type: {soil_type}, PH: {ph}",
        "alerts": risk_info["warnings"],
        "message": (
            f"It is {month_name} in {location_name}. "
            f"Current conditions: {temperature:.1f}°C, {rainfall:.1f}mm rain, {humidity:.1f}% humidity. "
            f"{viability_msg if user_crop else f'Our AI suggests planting {prediction}.'} "
            f"Current risk level is {risk_info['level']}."
        )
    }
    return advice
print("GENERAL ADVICE")
print(farmer_advice(-0.42, 36.95, "loamy", 90, 40, 40, 6.5))

print("SPECIFIC CROP (rice)")
print(farmer_advice(-0.42, 36.95, "loamy", 90, 40, 40, 6.5, user_crop="rice"))
joblib.dump(model, "farmer_advisory_model.pkl")   